In [1]:
from pl_modules import HuggingfaceDatasetWrapper, PlLanguageModelForSequenceOrdering, so_data_collator
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForTokenClassification
from datasets import load_from_disk

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = PlLanguageModelForSequenceOrdering.load_from_checkpoint('/mnt/data/users/keller/Uni/trf_training_tut/scripts/pytorch-lightning/rocstories/lightning_logs/version_0/checkpoints/epoch=0-step=4999.ckpt')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [2]:
data = load_from_disk("../data/rocstories")

# Downsampling for debugging...
# data = data.filter(lambda _, index: index < 100, with_indices=True)

dataset = HuggingfaceDatasetWrapper(
    data,
    text_column="text",
    target_column="so_targets",
    tokenizer=tokenizer,
    mapping_funcs=[],
    collate_fn=so_data_collator,
)

In [7]:
from pytorch_lightning import Trainer

trainer = Trainer(gpus=1, auto_lr_find=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [8]:
trainer.tune(model=model, datamodule=dataset)

MisconfigurationException: When `auto_lr_find=True`, either `model` or `model.hparams` should have one of these fields: ('lr', 'learning_rate') overridden.

In [5]:
trainer.test(model=model, datamodule=dataset)

  0%|          | 0/43 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/keller/.conda/envs/huggingface_latest/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': 2.719205141067505,
 'loss_epoch': 2.719205141067505,
 'train_loss': 2.719205141067505,
 'train_loss_epoch': 2.719205141067505,
 'val_acc': 0.6673813462257385,
 'val_acc_epoch': 0.6673813462257385,
 'val_kendalls_tau': 0.7670419812202454,
 'val_kendalls_tau_epoch': 0.7670419812202454,
 'val_mean_logits': 2.127222776412964,
 'val_mean_logits_epoch': 2.127222776412964,
 'val_std_logits': 1.0633916854858398,
 'val_std_logits_epoch': 1.0633916854858398}
--------------------------------------------------------------------------------


[{'train_loss': 2.719205141067505,
  'train_loss_epoch': 2.719205141067505,
  'val_kendalls_tau': 0.7670419812202454,
  'val_kendalls_tau_epoch': 0.7670419812202454,
  'val_acc': 0.6673813462257385,
  'val_acc_epoch': 0.6673813462257385,
  'val_mean_logits': 2.127222776412964,
  'val_mean_logits_epoch': 2.127222776412964,
  'val_std_logits': 1.0633916854858398,
  'val_std_logits_epoch': 1.0633916854858398,
  'loss': 2.719205141067505,
  'loss_epoch': 2.719205141067505}]

In [10]:
tokenizer.cls_token_id

101